# **Multiple Cryptographic Functionalities**

---







In [ ]:
#Name: Alina Hasan
#ID: 20101301

### Write a program that integrates multiple cryptographic functionalities into a single application. You are allowed to import cryptographic libraries and use built-in methods.

# **Install & Import**

---



In [ ]:
!pip install pycryptodome

In [ ]:
import os
import sys
import ast
import hmac
import hashlib
import random
import binascii
import base64
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Util import number
from Crypto.Hash import SHA256
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Util.Padding import pad, unpad
from cryptography.hazmat.primitives import cmac
from cryptography.hazmat.primitives.ciphers import algorithms

# **Other Functions**

---



In [ ]:
def check_option(options, prompt):

        while True:

            choice = input(prompt)

            if choice in options:

                return choice

            else:

                print("Invalid choice. Please choose a valid option.")

def is_prime(n):

        if n > 1:

            for i in range(2, int(n / 2) + 1):

                if (n % i) == 0:

                    return 0

                else:

                   return 1

        else:

            return 0

def check_prime(prompt):

    while True:

        n = int(input(prompt))

        if is_prime(n) == 0:

            print(f"{n} is not a prime number")

        else:

            return n

def is_coprime(a, b): #OR,  GCD

    if a == 0 or b == 0:

        return 0

    if a == b:

        return a

    if a > b:

        return is_coprime(a - b, b)

    return is_coprime(a, b - a)


def check_coprime(a, prompt):

    while True:

        b = int(input(prompt))

        if is_coprime(a, b) == 1:

            return b

        else:

            print(f"{b} is not relatively prime to {a}")

### The program should allow users to perform the following operation:

# **1. AES Encryption/Decryption**
Should support AES encryption and decryption using either ECB or CBC mode. Users should be able to input the plaintext or ciphertext, specify the AES mode, provide the necessary keys, and select the operation (encryption or decryption). The program should deliver the expected output based on the user's input.

**Required inputs:** Mode (ECB/CBC), Operation (Encrypt/Decrypt), Plaintext/Ciphertext, Key

---



In [ ]:
class AESDemo:

    def encrypt(self, mode, message, key):

        message = pad(bytes(message, 'utf-8'), AES.block_size) #Ensures size is 128-bits
        key = pad(bytes(key, 'utf-8'), AES.block_size)

        if mode == '1':

            IV = Random.new().read(AES.block_size)
            AES_obj = AES.new(key, AES.MODE_CBC, IV)
            ciphertext = IV + AES_obj.encrypt(message)

        elif mode == '2':

            AES_obj = AES.new(key, AES.MODE_ECB)
            ciphertext = AES_obj.encrypt(message)

        return ciphertext

    def decrypt(self, mode, message, key):

        message = ast.literal_eval(message)
        key = pad(bytes(key, 'utf-8'), AES.block_size)

        if mode == '1':

            IV,  message = message[:AES.block_size], message[AES.block_size:]
            AES_obj = AES.new(key, AES.MODE_CBC, IV)
            message = AES_obj.decrypt(message)

        elif mode == '2':

            AES_obj = AES.new(key, AES.MODE_ECB)
            message = AES_obj.decrypt(message)

        message = unpad(message, AES.block_size).decode('utf-8')

        return message

    def run_demo(self):

        options = ['1', '2']

        print("1. CBC")
        print("2. ECB")
        mode = check_option(options, "Select mode (1/2): ")

        print("1. Encrypt")
        print("2. Decrypt")
        operation = check_option(options, "Select operation (1/2): ")

        message = input("Enter message: ")
        key = input("Enter Key: ")

        if operation == '1':

            ciphertext = self.encrypt(mode, message, key)
            print("Encrypted text:", ciphertext)

        else:

            plaintext = self.decrypt(mode, message, key)
            print("Decrypted text:", plaintext)

# Example usage
# demo = AESDemo()
# demo.run_demo()

# **2. RSA Encryption/Decryption**

Should offer RSA encryption and decryption capabilities. Users should be able to input the plaintext or ciphertext, select the RSA operation, provide the required RSA keys, and choose the desired encryption or decryption process.

**Required inputs:** Operation (Encrypt/Decrypt), Plaintext/Ciphertext, Key

---



In [ ]:
class RSADemo:

    def generate_keys(self):

        print("Generating Keys...")

        p = int(check_prime("Enter a prime number: "))
        q = int(check_prime("Enter another prime number: "))

        N = p * q
        phi = (p - 1) * (q - 1)

        e = int(check_coprime(phi, f"Enter a relatively prime number to {phi}: "))

        d = pow(e, -1, phi)

        return N, e, phi, d

    def encrypt(self, M, N, e):

        C = pow(M, e, N)
        # C = M ** e % N

        return C

    def decrypt(self, C, N, d):

        M = pow(C, d, N)
        # M = C ** d % N

        return M

    def prompt_encrypt(self, N, e, d):

        M = int(input("Enter message: "))

        C = self.encrypt(M, N, e)
        print(f"Encrypted message: {C}")
        print(f"Corresponding public key (N,e): {(N, e)}")
        print(f"Corresponding private key: {d} ")

        return M, C


    def run_demo(self):

        options = ['1', '2']

        print("1. Encrypt")
        print("2. Decrypt")

        operation = check_option(options, "Select operation (1/2): ")

        if operation == '1':

            N, e, phi, d = self.generate_keys()
            self.prompt_encrypt(N, e, d)

        elif operation == '2':

            C = int(input("Enter encrypted message: "))
            N = int(input("Enter public key, N: "))
            d = int(input("Enter private key: "))

            M = self.decrypt(C, N, d)
            print("Decrypted message: ", M)

        else:

            print("Invalid choice. Select operation (1/2).")

# Example Usage
# demo = RSADemo()
# demo.run_demo()

#TRY FOR ACTUAL MESSAGES USING ASCII ord() and chr()

# **3. Hashing**

Should include hashing functionalities supporting SHA1 and SHA256 algorithms. Users should be able to input the data to be hashed, specify the hash mode (SHA1 or SHA256), and trigger the operation. The program should generate the corresponding hash value.

**Required inputs:** Plaintext, Hash Mode(SHA1/SHA256/MD5)

---




In [ ]:
class HashDemo:

    def hash(self, mode, plaintext):

        plaintext = plaintext.encode('utf-8')

        if mode == '1':

            return hashlib.sha1(plaintext)

        elif mode == '2':

            return hashlib.sha256(plaintext)

        else:

            return  hashlib.md5(plaintext)

    def run_demo(self):

        options = ['1', '2', '3']

        print("1. SHA1")
        print("2. SHA256")
        print("3. MD5")

        mode = check_option(options, ("Select hash mode (1/2/3): "))

        message = input("Enter message: ")
        hashed_value = self.hash(mode, message).hexdigest()
        print(f"Corresponding hash value: {hashed_value}")

#Example Usage
# demo = HashDemo()
# demo.run_demo()

#**4. Digital Signature using RSA**

Should facilitate the generation and verification of digital signatures using RSA. Users should be able to input the message to be signed, select the RSA keys for signing, and initiate the signing process. Similarly, users should be able to input the signed message, specify the RSA public key, and perform the digital signature verification. The program should ensure the integrity and authenticity of the digital signatures.

**Required inputs:** Operation(Generation/Verification), Message/Signature


---



In [ ]:
#With Hash

class DigitalSignatureRSA:

    def generate_keys(self):

        key = RSA.generate(1024)

        private_key = base64.b64encode(key.export_key()).decode('utf-8')
        public_key = base64.b64encode(key.publickey().export_key()).decode('utf-8')

        return private_key, public_key

    def generate_signature(self, private_key, message):

        key = RSA.import_key(base64.b64decode(private_key))
        hashed_value = SHA256.new(message.encode('utf-8'))
        signature = pkcs1_15.new(key).sign(hashed_value)

        return signature

    def verify_signature(self, public_key, message, signature):

        key = RSA.import_key(base64.b64decode(public_key))
        hashed_value = SHA256.new(message.encode('utf-8'))

        try:

            pkcs1_15.new(key).verify(hashed_value, signature)

            return True

        except:

            return False

    def run_demo(self):

        options = ['1', '2']
        print("1. Generate Signature")
        print("2. Verify Signature")
        operation = check_option(options, "Select operation (1/2): ")

        if operation == '1':

            private_key, public_key = self.generate_keys()
            message = input("Enter message: ")
            signature = self.generate_signature(private_key, message)
            print(f"Generated Signature: {base64.b64encode(signature).decode('utf-8')}")
            print(f"Corresponding private key: {private_key}")
            print(f"Corresponding public key: {public_key}")

        elif operation == '2':

            public_key = input("Enter Public Key: ")
            message = input("Enter message: ")
            signature = input("Enter signature: ")
            signature = base64.b64decode(signature)
            valid  = self.verify_signature(public_key, message, signature)

            if valid:

                print("Signature Valid.")

                return

            print("Signature Invalid.")


#Example Usage
# demo = DigitalSignatureRSA()
# demo.run_demo()

#**5. MAC Generation:**

The program should allow users to generate Message Authentication Codes (MAC) for data integrity and authenticity. Users should be able to input the data, select the MAC algorithm, and initiate the MAC generation process. The program should generate the MAC value accurately, maintaining the security of the data.

**Required inputs:** Message

---



In [ ]:
class MACDemo:

    def CBC_MAC(self, message, key): #Cipher Block Chaining-MAC

        c_obj = cmac.CMAC(algorithms.TripleDES(key))
        c_obj.update(message)
        signature = c_obj.finalize()

        print (f"Message: {message.decode()}" )
        print (f"CMAC signature: {binascii.b2a_hex(signature).decode()}")

    def Hash_MAC(self, message, key):

        signature = hmac.new(key, message, hashlib.sha256).hexdigest()
        print(f"HMAC (SHA-256) signature: {signature}")

        return signature

    def run_demo(self):

        options = ['1', '2']
        print('1. CMAC')
        print('2. HMAC')
        operation = check_option(options, "Select operation (1/2): ")

        message = input("Enter message: ").encode()
        key = os.urandom(16)

        if operation == '1':

            self.CBC_MAC(message, key)

        if operation == '2':

            # key = input("Enter Key: ").encode()
            self.Hash_MAC(message, key)

#Example Usage
# mac_demo = MACDemo()
# mac_demo.run_demo()

In [ ]:
# #verify the given information here
# MAC("I met an interesting turtle while the song on the radio blasted away", b'\x01\xd8i\xa1^0\x9a<\x0f\xf0\r\xc1\xdd\xd5\x89\xa6')
#ba4ecb8db45c6ae0
# MAC("I like to leave work after my eight-hour tea-break", b'\xa6+\x16\x9d-1\xda\x8aV\xed\xf5\xf0cv\x04\x88')
5# MAC("Her daily goal was to improve on yesterday", b'[\xc5\xbd\xe4z\xd1=E\x17-ku\x02=|=')
# MAC("He found the chocolate covered roaches quite tasty", b'5"k\xff\x81a\x9b 7\x8c>\xb7\xb9\xdcu\xaa')
# MAC("After fighting off the alligator, Brian still had to face the anaconda", b'\xa1\xfcw"?3\x91\x1c\t\x9c\x91\xe2He\x935')
# MAC("He decided to count all the sand on the beach as a hobby", b'\xa7\x83@\xde\xbf\xb494\xee\x84\x1e-\xc8A\xf9:')
# MAC("The sign said there was road work ahead so he decided to speed up", b'2\xcbv\xdcU6\x99\xb6.\xa7\xea\xeb\xaf\x10\xc7\x90')
# MAC("Send 500$ to this account - 6589415651548", b'\xc3\xea\x99e\xaal\xab\xd4\x9b\xf9\xb4Z\x19\xed\xcf\xcb')
# MAC("Garlic ice-cream was her favorite", b'\x05\xf9\x83\x9d\xb7\xb6\xc3\xb8\x9e\xc5\xd9\xd8\x07]\xc6\xb3')
# MAC("I'd rather be a bird than a fish", b'\x84YY\xf0\x02GU\xa4LD\xd5\x85!A\xc2c')

5

# **Implementation**

---



In [ ]:
while True:

    options = ['1', '2', '3', '4', '5', '6' ]

    print("1. AES Encryption/Decryption")
    print("2. RSA Encryption/Decryption")
    print("3. Hashing")
    print("4. Digital Signature using RSA")
    print("5. MAC Generation")
    print("6. Exit")

    operation = check_option(options, "Select Option 1~6: ")

    if operation == '1':

        demo = AESDemo()
        demo.run_demo()

    elif  operation == '2':

        demo = RSADemo()
        demo.run_demo()

    elif operation == '3':

        demo = HashDemo()
        demo.run_demo()

    elif operation == '4':

        demo = DigitalSignatureRSA()
        demo.run_demo()

    elif operation == '5':

        demo = MACDemo()
        demo.run_demo()

    elif operation == '6':

        print("Exiting...")

        break

    print("\n-----------------------------------------------------------------------------\n")


1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Select Option 1~6: 2
1. Encrypt
2. Decrypt
Select operation (1/2): 1
Generating Keys...
Enter a prime number: 29
Enter another prime number: 31
Enter a relatively prime number to 840: 14
14 is not relatively prime to 840
Enter a relatively prime number to 840: 11
Enter message: 123
Encrypted message: 371
Corresponding public key (N,e): (899, 11)
Corresponding private key: 611 

-----------------------------------------------------------------------------

1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Select Option 1~6: 2
1. Encrypt
2. Decrypt
Select operation (1/2): 2
Enter encrypted message: 371
Enter public key, N: 899
Enter private key: 611
Decrypted message:  123

-----------------------------------------------------------------------------

1. AES Encryption/Decryption
2. RSA En